잡플래닛에서 기업의 평점을 수집해 가공해보기<br>
평가 점수와 날짜만 수집.<br>
(기회가 된다면 XPATH, CSS_SELECTOR 둘 중 하나로 통일하기) 

In [67]:
import selenium
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

base_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(base_dir, "chromedriver.exe")
service = Service(file_path)
driver = webdriver.Chrome(service=service)

In [68]:
corp = "SK하이닉스"
url = "https://www.jobplanet.co.kr/companies/20561/reviews/%EC%97%90%EC%8A%A4%EC%BC%80%EC%9D%B4%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4"
driver.get(url)
driver.implicitly_wait(10) # 페이지가 전부 로딩되기까지 최대 10초 대기

try:
    driver.find_element(By.CSS_SELECTOR, "#i8zsl").click()  # ID로 찾기
except:
    pass

In [69]:
review_list = driver.find_elements(By.CSS_SELECTOR, "#viewReviewsList")
review = review_list[0]
score_list = review.find_element(By.CSS_SELECTOR, r"#ReviewCardSide > div.mb-\[32px\] > span")
date = review.find_elements(By.CSS_SELECTOR, "#review_4987244 > div:nth-child(1) > div.relative.flex.items-center.justify-between > div > span:nth-child(7)")[0].get_attribute("innerHTML")[:-3]
print(date)

2025. 03


In [70]:
score_list = review.find_element(By.CSS_SELECTOR, "#ReviewCardSide")
star_score = score_list.find_element(By.CSS_SELECTOR, r"#ReviewCardSide > div.mb-\[32px\] > span").get_attribute("innerHTML")
display(star_score)

'5.0'

In [75]:
# 항목별 평점
bl_score_list = score_list.find_elements(By.CSS_SELECTOR, "#ReviewCardSide > div:nth-child(2) > ul")
# promotion_opp = len(score_list.find_element(By.XPATH, ".//p[contains(text(), '승진 기회')]/../div/ul").find_elements(By.XPATH, '//*[@id="ReviewCardSide"]/div[2]/ul/li[5]/div/ul/li[4]/span'))
promotion_opp = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '승진 기회')]/../div//span[contains(@style, 'width: 100%')]"
))
welfare_salary = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '복지/급여')]/../div//span[contains(@style, 'width: 100%')]"
))
work_life_balance = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '워라밸')]/../div//span[contains(@style, 'width: 100%')]"
))
culture = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '사내문화')]/../div//span[contains(@style, 'width: 100%')]"
))
management = len(score_list.find_elements(
    By.XPATH,
    ".//p[contains(text(), '경영진')]/../div//span[contains(@style, 'width: 100%')]"
))
print(promotion_opp, welfare_salary, work_life_balance, culture, management)
other_score = [promotion_opp, welfare_salary, work_life_balance, culture, management]

5 5 5 5 4


In [80]:
# 한 리뷰에서 전체 점수를 가져와서 정제하는 함수를 만들자. 위 코드들을 적절히 합친다.
def extract_score_from_review(review): 
    # 출력 초기화
    output = []
    # 리뷰에서 평점과 관련된 부분만 가져오기
    score_list = review.find_element(By.CSS_SELECTOR, "#ReviewCardSide")
    # 총 평점을 가져와서 추가
    star_score = score_list.find_element(By.CSS_SELECTOR, r"#ReviewCardSide > div.mb-\[32px\] > span").get_attribute("innerHTML")
    output.append(star_score)
    # 항목별 평점을 가져와서 추가
    other_score_elements = score_list.find_elements(By.CSS_SELECTOR, "#ReviewCardSide > div:nth-child(2) > ul")
    for element in other_score_elements:
        output.append(other_score)
    return output
    
data = extract_score_from_review(review)
result = [data[0]] + data[1]
print(result)

['5.0', 5, 5, 5, 5, 4]


In [ ]:
# 전체 페이지 수는 알 수 없음. 
# 다음 페이지로 이동하는 함수를 만들자.
import time
def get_url(driver, url):
    while True:
        try:
            driver.get(url)
            driver.implicitly_wait(10)
            break
        except:
            driver.close()
            driver = webdriver.Chrome(service=service)
            time.sleep(60)

In [84]:
import pandas as pd
base_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(base_dir, "data", "종목별_잡플래닛_CHO.xlsx")
input_data = pd.read_excel(file_path)
data = []

for company, url in input_data[['종목명', 'URL']].values:
    # 초기 URL 들어가기
    get_url(driver, url)
    # 페이지 순회 시작
    for page in range